In [ ]:
import pickle
import mne
import numpy as np
from collections import Counter
import networkx as nx
from collections import defaultdict
from sklearn.manifold import SpectralEmbedding
import matplotlib.pyplot as plt
import matplotlib
import pickle
import numpy as np
from scipy.spatial.distance import pdist, squareform
from scipy.stats import entropy
from sklearn.decomposition import PCA
import seaborn as sns
import pathlib
from tqdm import tqdm
from mne.preprocessing import ICA
from scipy.signal import butter, filtfilt
from pathlib import Path

In [ ]:
# Sampling rate
sfreq = 500  # Hz

# Load your datasets
with open("/home/donaf-strange/LAB_WORK/eeg_arithmetic_project/data/all_rest_eeg_by_participant.pkl", 'rb') as f:
    rest_eeg_dict = pickle.load(f)

with open("/home/donaf-strange/LAB_WORK/eeg_arithmetic_project/data/all_task_eeg_by_participant.pkl", 'rb') as f:
    task_eeg_dict = pickle.load(f)

# Define mapping for EEG channels (from raw names to standard ones)
channel_mapping = {
    "EEG Fp1": "Fp1", "EEG Fp2": "Fp2", "EEG F3": "F3", "EEG F4": "F4",
    "EEG F7": "F7", "EEG F8": "F8", "EEG T3": "T3", "EEG T4": "T4",
    "EEG C3": "C3", "EEG C4": "C4", "EEG T5": "T5", "EEG T6": "T6",
    "EEG P3": "P3", "EEG P4": "P4", "EEG O1": "O1", "EEG O2": "O2",
    "EEG Fz": "Fz", "EEG Cz": "Cz", "EEG Pz": "Pz"
}

# Preprocessing for one subject
def preprocess_subject(raw: mne.io.Raw, subject_id=None, label='UNKNOWN', n_components=0.99):
    raw = raw.copy()

    # Rename EEG channels
    raw.rename_channels(lambda ch: ch.replace("EEG ", "") if ch.startswith("EEG ") else ch)

    # Keep only EEG channels
    raw.pick(list(channel_mapping.values()))

    # Set average reference
    raw.set_eeg_reference('average', projection=False)

    # ICA - fit on high-pass filtered copy only (NOT applied)
    raw_ica = raw.copy().filter(l_freq=1.0, h_freq=None, fir_design='firwin', verbose=False)
    ica = ICA(n_components=n_components, random_state=97, max_iter='auto')
    ica.fit(raw_ica)

    # Apply ICA to full-band raw signal
    raw = ica.apply(raw)
    print(f"📊 Subject {subject_id} | {label} | ICA components used: {ica.n_components_}")

    # Frequency bands
`    bands = {
        #'delta': (1, 4),
        'theta': (4, 8),
        'alpha': (8, 12),
        'beta': (12, 30),
        'gamma': (30, 45),
        'broadband': (1, 45)
    }
`

    # Band-specific filtered data
    band_data = {}
    for band, (low, high) in bands.items():
        raw_band = raw.copy().filter(l_freq=low, h_freq=high, fir_design='firwin', verbose=False)
        band_data[band] = raw_band.get_data()

    return band_data


# Batch preprocessing
def batch_preprocess(eeg_dict, label='UNKNOWN'):
    preprocessed = {}
    for subject_id, raw in eeg_dict.items():
        try:
            band_data = preprocess_subject(raw, subject_id=subject_id, label=label)
            preprocessed[subject_id] = band_data
            print(f"✅ Preprocessed {label} subject {subject_id}")
        except Exception as e:
            print(f"❌ Failed {label} subject {subject_id}: {e}")
    return preprocessed





In [ ]:
# Save outputs for Step 2
Path("preprocessed").mkdir(exist_ok=True)

In [ ]:
preprocessed_task = batch_preprocess(task_eeg_dict, label='TASK')

In [ ]:
with open("preprocessed/preprocessed_task.pkl", "wb") as f:
    pickle.dump(preprocessed_task, f)

In [ ]:
preprocessed_rest = batch_preprocess(rest_eeg_dict, label='REST')

In [ ]:
with open("preprocessed/preprocessed_rest.pkl", "wb") as f:
    pickle.dump(preprocessed_rest, f)

In [ ]:

def binarize_subject_band(band_data, method='median'):
    binary_band = {}

    for band, data in band_data.items():
        n_channels, n_samples = data.shape
        binary = np.zeros_like(data, dtype=np.uint8)

        for ch in range(n_channels):
            if method == 'median':
                threshold = np.median(data[ch])
            elif method == 'quantile':
                threshold = np.quantile(data[ch], 0.5)
            else:
                raise ValueError("Unsupported binarization method.")

            binary[ch] = (data[ch] > threshold).astype(np.uint8)

        binary_band[band] = binary

    return binary_band

In [ ]:
def batch_binarize(preprocessed_dict, label='UNKNOWN'):
    binary_all = {}
    for subj_id, band_data in preprocessed_dict.items():
        try:
            binary_all[subj_id] = binarize_subject_band(band_data)
            print(f"✅ Binarized {label} subject {subj_id}")
        except Exception as e:
            print(f"❌ Failed {label} subject {subj_id}: {e}")
    return binary_all

In [ ]:
# Load preprocessed filtered data
with open("preprocessed/preprocessed_rest.pkl", "rb") as f:
    preprocessed_rest = pickle.load(f)

with open("preprocessed/preprocessed_task.pkl", "rb") as f:
    preprocessed_task = pickle.load(f)

# Run
binary_rest = batch_binarize(preprocessed_rest, label='REST')
binary_task = batch_binarize(preprocessed_task, label='TASK')

# Save
Path("binarized").mkdir(exist_ok=True)
with open("binarized/binary_rest.pkl", "wb") as f:
    pickle.dump(binary_rest, f)

with open("binarized/binary_task.pkl", "wb") as f:
    pickle.dump(binary_task, f)


In [ ]:
def fit_pairwise_ising(X):
    """
    Fit Ising model using logistic regression (pseudo-likelihood method).
    Input: X ∈ {0,1}^n_samples × n_channels
    Output: h ∈ R^n, J ∈ R^{n × n}
    """
    X = X.copy().astype(int)  # Fix: convert from uint8 → int
    X[X == 0] = -1  # Convert 0 → -1
    n_samples, n_channels = X.shape

    h = np.zeros(n_channels)
    J = np.zeros((n_channels, n_channels))

    for i in range(n_channels):
        X_i = X[:, i]
        X_rest = np.delete(X, i, axis=1)

        model = LogisticRegression(penalty=None, solver='lbfgs', max_iter=1000)
        model.fit(X_rest, X_i)

        h[i] = model.intercept_[0]
        coefs = model.coef_[0]
        idx = 0
        for j in range(n_channels):
            if j == i:
                continue
            J[i, j] = coefs[idx]
            idx += 1

    # Symmetrize J
    J = 0.5 * (J + J.T)

    return h, J


In [ ]:
def fit_ising_batch(binary_data):
    ising_models = {}
    for subj_id in tqdm(binary_data):
        ising_models[subj_id] = {}
        for band, binary in binary_data[subj_id].items():
            X = binary.T  # shape: (timepoints, channels)
            h, J = fit_pairwise_ising(X)
            ising_models[subj_id][band] = {
                'h': h,
                'J': J
            }
    return ising_models

In [ ]:
from sklearn.linear_model import LogisticRegression

# Load binarized data
with open("binarized/binary_rest.pkl", "rb") as f:
    binary_rest = pickle.load(f)

with open("binarized/binary_task.pkl", "rb") as f:
    binary_task = pickle.load(f)

# Run fitting
ising_rest = fit_ising_batch(binary_rest)
ising_task = fit_ising_batch(binary_task)

# Save results
Path("ising_models").mkdir(exist_ok=True)
with open("ising_models/ising_rest.pkl", "wb") as f:
    pickle.dump(ising_rest, f)

with open("ising_models/ising_task.pkl", "wb") as f:
    pickle.dump(ising_task, f)


In [ ]:
from itertools import product

def all_binary_states(n):
    """Return all 2^n binary states of length n as np.array of shape (2^n, n)"""
    return np.array(list(product([1, -1], repeat=n)))  # use {-1, +1}


Compute Energy
E(s)=−h^⊤*s−s^⊤*J*s

In [ ]:
def compute_energies(states, h, J):
    linear = - np.dot(states, h)
    quad = - np.einsum('ij,ij->i', np.dot(states, J), states)
    return linear + quad  # shape: (2^n,)

In [ ]:
def hamming_neighbors(index, n_bits):
    """
    Return indices of neighbors of a binary state by flipping one bit.
    Input: index ∈ [0, 2^n), n_bits = number of bits (channels)
    Output: list of neighbor indices
    """
    return [index ^ (1 << i) for i in range(n_bits)]

def find_local_minima(energies):
    n_states = len(energies)
    n_bits = int(np.log2(n_states))
    minima = []
    for i in range(n_states):
        neighbors = hamming_neighbors(i, n_bits)
        if all(energies[i] < energies[j] for j in neighbors):
            minima.append(i)
    return minima


Combined Function to Run Per Subject/Band

In [ ]:

def extract_landscape_features(h, J):
    """
    Computes energy landscape features given Ising model parameters h and J.
    Includes:
        - num_minima
        - minima_indices
        - min_energy
        - avg_energy
        - energy_range
        - energy_entropy (Boltzmann from normalized energies)
    """
    n = len(h)
    states = all_binary_states(n)  # shape: (2^n, n)
    energies = compute_energies(states, h, J)  # shape: (2^n,)
    
    minima_indices = find_local_minima(energies)
    num_minima = len(minima_indices)

    # Normalize energies
    norm_energies = (energies - energies.min()) / (energies.max() - energies.min())
    probs = np.exp(-norm_energies)
    probs /= probs.sum()

    return {
        'num_minima': num_minima,
        'minima_indices': minima_indices,
        'min_energy': float(np.min(energies)),
        'avg_energy': float(np.mean(energies)),
        'energy_range': float(np.max(energies) - np.min(energies)),
        'energy_entropy': float(entropy(probs))
    }


In [ ]:
def batch_extract_features(ising_dict):
    all_features = {}
    for subj_id in tqdm(ising_dict):
        all_features[subj_id] = {}
        for band in ising_dict[subj_id]:
            h = ising_dict[subj_id][band]['h']
            J = ising_dict[subj_id][band]['J']
            all_features[subj_id][band] = extract_landscape_features(h, J)
    return all_features


In [ ]:
with open("ising_models/ising_rest.pkl", "rb") as f:
    ising_rest = pickle.load(f)

with open("ising_models/ising_task.pkl", "rb") as f:
    ising_task = pickle.load(f)

# Run feature extraction
features_rest = batch_extract_features(ising_rest)
features_task = batch_extract_features(ising_task)

# Save
Path("landscape_features").mkdir(exist_ok=True)
with open("landscape_features/features_rest.pkl", "wb") as f:
    pickle.dump(features_rest, f)

with open("landscape_features/features_task.pkl", "wb") as f:
    pickle.dump(features_task, f)

In [ ]:
# Check number of subjects
print("✅ Number of subjects:", len(features_task))

# Check number of features for first subject
first_subj = list(features_task.keys())[0]
first_features = features_task[first_subj]

# If it's nested (e.g., by band), flatten and count
flat_features = {}

for band, band_feats in first_features.items():
    for feat_name in band_feats:
        flat_features[f"{band}_{feat_name}"] = band_feats[feat_name]

print("✅ Total number of features for one subject:", len(flat_features))
print("📋 Sample feature names:", list(flat_features.keys())[:10])

In [ ]:
# Good/Bad group definitions using numeric keys as strings
bad_counters = ['0', '4', '6', '9', '10', '14', '19', '21', '22', '30']
good_counters = ['1', '2', '3', '5', '7', '8', '11', '12', '13', '15', '16', '17',
                 '18', '20', '23', '24', '25', '26', '27', '28', '29', '31', '32',
                 '33', '34', '35']


In [ ]:
def aggregate_group_features(features_dict, subjects, band='alpha'):
    summary_list = []
    for sid in subjects:
        if sid in features_dict and band in features_dict[sid]:
            summary = summarize_features(features_dict[sid][band])
            summary_list.append(summary)
    return summary_list


In [ ]:
from scipy.stats import ttest_ind, mannwhitneyu

def compare_groups(good, bad, feature_name):
    good_vals = [s[feature_name] for s in good]
    bad_vals = [s[feature_name] for s in bad]

    # Non-parametric test
    stat, pval = mannwhitneyu(good_vals, bad_vals, alternative='two-sided')

    print(f"\n📊 Feature: {feature_name}")
    print(f"  Mean (Good): {np.mean(good_vals):.3f}")
    print(f"  Mean (Bad) : {np.mean(bad_vals):.3f}")
    print(f"  Mann-Whitney U test: p = {pval:.4f}")


In [ ]:
# Load features
with open("landscape_features/features_rest.pkl", "rb") as f:
    features_rest = pickle.load(f)

# Choose band
band = 'theta'

# Group-wise summaries
summary_good = aggregate_group_features(features_rest, good_counters, band=band)
summary_bad = aggregate_group_features(features_rest, bad_counters, band=band)

# Compare features
compare_groups(summary_good, summary_bad, 'num_minima')
compare_groups(summary_good, summary_bad, 'energy_entropy')
compare_groups(summary_good, summary_bad, 'min_energy')


In [ ]:
import pandas as pd

In [ ]:
def plot_feature_distribution(good, bad, feature_name, band):
    df = pd.DataFrame({
        'Value': [s[feature_name] for s in good] + [s[feature_name] for s in bad],
        'Group': ['Good'] * len(good) + ['Bad'] * len(bad),
        'Feature': feature_name
    })

    plt.figure(figsize=(6, 4))
    sns.boxplot(x='Group', y='Value', data=df, palette='Set2')
    plt.title(f"{feature_name} - {band} band")
    plt.tight_layout()
    plt.show()

In [ ]:
plot_feature_distribution(summary_good, summary_bad, 'energy_entropy', band)

In [ ]:
def plot_energy_histogram(features_dict, subject_id, band='alpha', normalized=False):
    energies = features_dict[subject_id][band]['normalized_energies'] if normalized \
               else features_dict[subject_id][band]['energies']
    
    plt.figure(figsize=(6, 3))
    plt.hist(energies, bins=100, color='slateblue', alpha=0.7)
    plt.title(f"{'Normalized' if normalized else 'Raw'} Energy Distribution\nSubject {subject_id} | Band: {band}")
    plt.xlabel("Energy")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.show()

In [ ]:
plot_energy_histogram(features_rest, subject_id='3', band='alpha', normalized=True)

In [ ]:
def plot_minima_energies(features_dict, subject_id, band='alpha'):
    all_energies = features_dict[subject_id][band]['energies']
    minima_indices = features_dict[subject_id][band]['minima_indices']
    minima_energies = all_energies[minima_indices]

    plt.figure(figsize=(5, 3))
    plt.hist(minima_energies, bins=20, color='darkorange', alpha=0.8)
    plt.title(f"Minima Energies\nSubject {subject_id} | Band: {band}")
    plt.xlabel("Energy")
    plt.ylabel("Number of Minima")
    plt.tight_layout()
    plt.show()


In [ ]:
plot_minima_energies(features_rest, subject_id='3', band='alpha')

In [ ]:
def print_feature_summary(features_dict, subject_id, band='alpha'):
    f = features_dict[subject_id][band]
    print(f"\n🧠 Subject {subject_id} | Band: {band}")
    print(f"  ▸ Num minima        : {f['num_minima']}")
    print(f"  ▸ Min energy        : {np.min(f['energies']):.3f}")
    print(f"  ▸ Energy entropy    : {entropy(np.exp(-f['normalized_energies']) / np.sum(np.exp(-f['normalized_energies']))):.3f}")
    print(f"  ▸ Avg energy        : {np.mean(f['energies']):.3f}")
    print(f"  ▸ Energy range      : {np.max(f['energies']) - np.min(f['energies']):.3f}")
    print(f"  ▸ # State samples   : {len(f['energies'])}")


In [ ]:
print_feature_summary(features_rest, subject_id='3', band='gamma')

In [ ]:
def get_group_features(features_dict, subject_ids, band):
    keys = ['num_minima', 'min_energy', 'energy_entropy', 'avg_energy', 'energy_range']
    all_features = {k: [] for k in keys}
    all_features['minima_indices_len'] = []

    for sid in subject_ids:
        try:
            band_feats = features_dict[sid].get(band, {})
            for k in keys:
                all_features[k].append(band_feats[k])

            # Convert minima_indices to a scalar (length)
            all_features['minima_indices_len'].append(len(band_feats['minima_indices']))
        except Exception as e:
            print(f"❌ Skipping subject {sid} for band {band}: {e}")

    return all_features


In [ ]:
from scipy.stats import mannwhitneyu

def compare_feature(feature, good_vals, bad_vals, band):
    g_mean = np.mean(good_vals)
    b_mean = np.mean(bad_vals)
    stat, pval = mannwhitneyu(good_vals, bad_vals, alternative='two-sided')
    
    print(f"📊 {feature.upper()} ({band}):")
    print(f"   Good mean: {g_mean:.3f} | Bad mean: {b_mean:.3f} | p = {pval:.4f}")
    return {'feature': feature, 'band': band, 'good_mean': g_mean, 'bad_mean': b_mean, 'p': pval}


In [ ]:
def full_group_comparison(features_dict, good_ids, bad_ids, bands=['theta', 'alpha', 'beta', 'gamma', 'broadband']):
    results = []
    for band in bands:
        good_feats = get_group_features(features_dict, good_ids, band)
        bad_feats = get_group_features(features_dict, bad_ids, band)

        for feature in good_feats:
            result = compare_feature(feature, good_feats[feature], bad_feats[feature], band)
            results.append(result)
    return results


In [ ]:
with open("landscape_features/features_task.pkl", "rb") as f:
    features_task = pickle.load(f)

# Run
results_task = full_group_comparison(features_task, good_counters, bad_counters)


In [ ]:
with open("landscape_features/features_rest.pkl", "rb") as f:
    features_rest = pickle.load(f)

# Run
results_rest = full_group_comparison(features_rest, good_counters, bad_counters)

In [ ]:
Path("all_features").mkdir(exist_ok=True)

# Save the dictionary for ML use
with open("all_features/Advance_features_rest.pkl", "wb") as f:
    pickle.dump(features_rest, f)

print("✅ Saved Advance features to all_features/Advance_features_rest.pkl")




In [ ]:
# Save the dictionary for ML use
with open("all_features/Advance_features_task.pkl", "wb") as f:
    pickle.dump(features_task, f)

print("✅ Saved Advance features to all_features/Advance_features_task.pkl")

In [ ]:
with open("landscape_features/features_task.pkl", "rb") as f:
    features_rest = pickle.load(f)

In [ ]:
import pymc as pm
import arviz as az

def bayesian_compare(good_vals, bad_vals, feature, band):
    with pm.Model() as model:
        mu_good = pm.Normal("mu_good", mu=0, sigma=10)
        mu_bad = pm.Normal("mu_bad", mu=0, sigma=10)
        sigma = pm.HalfNormal("sigma", sigma=10)
        
        y_good = pm.Normal("y_good", mu=mu_good, sigma=sigma, observed=good_vals)
        y_bad = pm.Normal("y_bad", mu=mu_bad, sigma=sigma, observed=bad_vals)
        
        diff = pm.Deterministic("diff", mu_good - mu_bad)
        trace = pm.sample(2000, tune=1000, target_accept=0.95, progressbar=False, return_inferencedata=True)
        '''trace = pm.sample(
        draws=2000,
        tune=1000,
        target_accept=0.95,        # ← More conservative, fewer divergences
        max_treedepth=15,          # ← Allows deeper exploration of the posterior
        return_inferencedata=True,
        progressbar=False
        )'''

    az.plot_posterior(trace, var_names=["diff"], ref_val=0)
    summary = az.summary(trace, var_names=["diff"], hdi_prob=0.95)

    print(f"\n📊 Bayesian comparison for {feature.upper()} in {band.upper()}:")
    print(summary)
    
    return {
        'feature': feature,
        'band': band,
        'diff_mean': summary.loc['diff', 'mean'],
        'hdi_2.5%': summary.loc['diff', 'hdi_2.5%'],
        'hdi_97.5%': summary.loc['diff', 'hdi_97.5%'],
        'prob_positive': (trace.posterior["diff"] > 0).mean().item()
    }

In [ ]:
def aggregate_group_features(features_dict, subjects, band):
    feature_list = []
    for sid in subjects:
        try:
            if band in features_dict[sid]:
                feature_list.append(extract_all_features(features_dict[sid][band]))
        except:
            continue
    return feature_list

def run_bayesian_analysis_all(features_dict, good_ids, bad_ids, bands=['theta', 'alpha', 'beta', 'gamma', 'broadband']):
    results = []
    for band in bands:
        good = aggregate_group_features(features_dict, good_ids, band)
        bad = aggregate_group_features(features_dict, bad_ids, band)

        if not good or not bad:
            continue

        for feature in good[0].keys():
            try:
                good_vals = [g[feature] for g in good]
                bad_vals = [b[feature] for b in bad]
                result = bayesian_compare(good_vals, bad_vals, feature, band)
                results.append(result)
            except Exception as e:
                print(f"❌ Skipped {feature} in {band}: {e}")
    return results


In [ ]:
def run_bayesian_analysis_all(features_dict, good_ids=good_counters, bad_ids=bad_counters, bands=['theta', 'alpha', 'beta', 'gamma']):
    results = []
    for band in bands:
        good_feats = get_group_features(features_dict, good_ids, band)
        bad_feats = get_group_features(features_dict, bad_ids, band)

        for feature in good_feats:
            try:
                good_vals = good_feats[feature]
                bad_vals = bad_feats[feature]
                result = bayesian_compare(good_vals, bad_vals, feature, band)
                results.append(result)
            except Exception as e:
                print(f"❌ Skipped {feature} in {band}: {e}")

    return results


In [ ]:
results = run_bayesian_analysis_all(features_rest, good_counters, bad_counters)

In [ ]:
results = run_bayesian_analysis_all(features_task, good_counters, bad_counters)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Input data
data = pd.DataFrame({
    'Band': ['theta', 'alpha', 'beta', 'gamma', 'broadband'] * 2,
    'Condition': ['Task'] * 5 + ['Rest'] * 5,
    'Good': [36.462, 33.231, 31.077, 30.385, 34.077,
             32.346, 30.962, 31.769, 31.923, 30.346],
    'Bad':  [42.500, 35.900, 35.200, 35.800, 32.300,
             35.300, 29.500, 28.900, 34.700, 25.500],
    'p_val': [0.0890, 0.6453, 0.7906, 0.1672, 0.7105,
              0.2088, 1.0000, 0.5711, 0.3462, 0.1839]
})

# Melt for violin plot
df_long = data.melt(id_vars=['Band', 'Condition', 'p_val'], 
                    value_vars=['Good', 'Bad'], 
                    var_name='Group', value_name='Num Minima')

# Plot violin
plt.figure(figsize=(12, 7))
sns.violinplot(data=df_long, x='Band', y='Num Minima', hue='Group',
               split=True, inner='box', palette='pastel', bw_adjust=0.2)

# Overlay red dots for Task, green for Rest
for i, band in enumerate(['theta', 'alpha', 'beta', 'gamma', 'broadband']):
    # Get task and rest values
    task_vals = data[(data['Band'] == band) & (data['Condition'] == 'Task')][['Good', 'Bad']].values.flatten()
    rest_vals = data[(data['Band'] == band) & (data['Condition'] == 'Rest')][['Good', 'Bad']].values.flatten()
    
    # Plot task dots in red
    plt.scatter([i - 0.15, i + 0.15], task_vals, color='red', label='Task' if i == 0 else "", zorder=5)
    
    # Plot rest dots in green
    plt.scatter([i - 0.15, i + 0.15], rest_vals, color='green', label='Rest' if i == 0 else "", zorder=5)

# Add horizontal p-value text
y_label_pos = 22
for i, band in enumerate(['theta', 'alpha', 'beta', 'gamma', 'broadband']):
    task_p = data[(data['Band'] == band) & (data['Condition'] == 'Task')]['p_val'].values[0]
    rest_p = data[(data['Band'] == band) & (data['Condition'] == 'Rest')]['p_val'].values[0]
    plt.text(i, y_label_pos, f"Task p = {task_p:.3f}", ha='center', fontsize=9, color='red', weight='bold')
    plt.text(i, y_label_pos - 1.5, f"Rest p = {rest_p:.3f}", ha='center', fontsize=9, color='green', weight='bold')

# Customize
#plt.title("Number of Minima across Frequency Bands (Task vs Rest)", fontsize=14)
plt.xlabel("Frequency Band")
plt.ylabel("Number of Attractor Minima")
plt.ylim(20, 45)
plt.legend(title="Group", loc="upper right")
plt.grid(axis='y', linestyle='--', alpha=0.3)

'''# Explanation box
plt.text(-0.25, 17.5, "Red = Task minima\nGreen = Rest minima",
         fontsize=10, bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round,pad=0.5'))'''

plt.tight_layout()
plt.show()


In [ ]:
import arviz as az
import numpy as np
import matplotlib.pyplot as plt

# Replace with your real PyMC3 posterior diff samples
posterior = {
    "Theta (Task)": np.random.normal(loc=-3.47, scale=3.66, size=4000),
    "Theta (Rest)": np.random.normal(loc=-1.16, scale=3.43, size=4000)
}

# Convert to InferenceData
idata = az.from_dict(posterior=posterior)

# Plot with HDI
az.plot_posterior(idata,
                  kind='hist',
                  hdi_prob=0.95,
                  rope=(-1, 1),  # e.g., define ROPE if meaningful
                  ref_val=0,
                  figsize=(8, 4),
                  textsize=12,
                  point_estimate='mean')

plt.suptitle("Bayesian Posterior for ΔNUM_MINIMA (Good - Bad)", fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import arviz as az
import matplotlib.pyplot as plt

# Simulate posterior samples
posterior_task = np.random.normal(loc=-3.465, scale=3.658, size=4000)
posterior_rest = np.random.normal(loc=-1.16, scale=3.427, size=4000)

idata = az.from_dict(posterior={
    "Theta_NUM_MINIMA_Task": posterior_task,
    "Theta_NUM_MINIMA_Rest": posterior_rest
})

# Plot with HDI and ROPE
az.plot_posterior(idata,
                  hdi_prob=0.95,
                  rope=(-1, 1),
                  ref_val=0,
                  figsize=(10, 4),
                  textsize=12)

plt.suptitle("📊 Bayesian Posterior: Num Minima Difference (Good - Bad)", fontsize=15)
plt.tight_layout()
plt.show()
